# Create a Vertex AI Datastore and Search Engine

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/search/create_datastore_and_search.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/create_datastore_and_search.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/search/create_datastore_and_search.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/create_datastore_and_search.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/create_datastore_and_search.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/create_datastore_and_search.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/53/X_logo_2023_original.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/create_datastore_and_search.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/search/create_datastore_and_search.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            

---

* Author(s): [Kara Greenfield](https://github.com/kgreenfield2)
* Created: 22 Nov 2023
* Updated: 31 Oct 2024

---

## Objective

This notebook shows how to create and populate a Vertex AI Search Datastore, how to create a search app connected to that datastore, and how to submit queries through the search engine.


Services used in the notebook:

- ✅ Vertex AI Search for document search and retrieval

## Install pre-requisites

If running in Colab install the pre-requisites into the runtime. Otherwise it is assumed that the notebook is running in Vertex AI Workbench.

In [ ]:
%pip install --upgrade --user -q google-cloud-discoveryengine

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages

import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


## Authenticate

If running in Colab authenticate with `google.colab.google.auth` otherwise assume that running on Vertex AI Workbench.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

## Configure notebook environment

In [1]:
from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine
import pandas as pd

PROJECT_ID = "sandbox-373102"  # @param {type:"string"}
LOCATION = "global"
BQ_DATASET = "documents"
BQ_TABLE_1 = "sandbox-373102-first"
BQ_TABLE_2 = "sandbox-373102-second"

EXPORT_BUCKET = 'jk-bq-export'

Set [Application Default Credentials](https://cloud.google.com/docs/authentication/application-default-credentials)

In [ ]:
!gcloud auth application-default login --project {PROJECT_ID}

In [2]:
from google.cloud import bigquery

def export_table_to_jsonl(bucket_name, dataset_name, table_name):
    client = bigquery.Client()
    destination_uri = "gs://{}/{}".format(bucket_name, f"{table_name}.jsonl")
    dataset_ref = bigquery.DatasetReference(PROJECT_ID, dataset_name)
    table_ref = dataset_ref.table(table_name)
    job_config = bigquery.job.ExtractJobConfig()
    job_config.destination_format = bigquery.DestinationFormat.NEWLINE_DELIMITED_JSON

    extract_job = client.extract_table(
        table_ref,
        destination_uri,
        job_config=job_config,
        # Location must match that of the source table.
        location="US",
    )  # API request
    return extract_job.result()  # Waits for job to complete.

In [3]:
export_table_to_jsonl(EXPORT_BUCKET, BQ_DATASET, BQ_TABLE_1)
pd.read_json(f'gs://{EXPORT_BUCKET}/{BQ_TABLE_1}.jsonl', lines=True)

,id,structData,content
0,f45bbdb8-a828-404c-989b-c89d48fb292a,"{'category': ['2020년'], 'title': '2020년 국민연금기금...","{'mimeType': 'application/pdf', 'uri': 'gs://j..."
1,faf429fe-b5dc-43ac-9796-6dc9b178b062,"{'category': ['2019년'], 'title': '2019년 국민연금기금...","{'mimeType': 'application/pdf', 'uri': 'gs://j..."


In [4]:
export_table_to_jsonl(EXPORT_BUCKET, BQ_DATASET, BQ_TABLE_2)
pd.read_json(f'gs://{EXPORT_BUCKET}/{BQ_TABLE_2}.jsonl', lines=True)

,id,structData,content
0,382177ed-4418-40f3-86c5-9a4200bf12a4,"{'category': ['2021년'], 'title': '2021년 국민연금기금...","{'mimeType': 'application/pdf', 'uri': 'gs://j..."


## Create and Populate a Datastore

In [5]:
def create_data_store(
    project_id: str, location: str, data_store_name: str, data_store_id: str
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DataStoreServiceClient(client_options=client_options)

    # Initialize request argument(s)
    data_store = discoveryengine.DataStore(
        display_name=data_store_name,
        industry_vertical=discoveryengine.IndustryVertical.GENERIC,
        content_config=discoveryengine.DataStore.ContentConfig.CONTENT_REQUIRED,
    )

    operation = client.create_data_store(
        request=discoveryengine.CreateDataStoreRequest(
            parent=client.collection_path(project_id, location, "default_collection"),
            data_store=data_store,
            data_store_id=data_store_id,
        )
    )

    # Make the request
    # The try block is necessary to prevent execution from halting due to an error being thrown when the datastore takes a while to instantiate
    try:
        response = operation.result(timeout=90)
    except:
        print("long-running operation error.")

In [6]:
# The datastore name can only contain lowercase letters, numbers, and hyphens
DATASTORE_NAME = "pension_report_first"
DATASTORE_ID_FIRST = f"{DATASTORE_NAME}-id"

#Need to create at least two data store for blended search
create_data_store(PROJECT_ID, LOCATION, DATASTORE_NAME, DATASTORE_ID_FIRST)

In [7]:
# The datastore name can only contain lowercase letters, numbers, and hyphens
DATASTORE_NAME = "pension_report_second"
DATASTORE_ID_SECOND = f"{DATASTORE_NAME}-id"

#Need to create at least two data store for blended search
create_data_store(PROJECT_ID, LOCATION, DATASTORE_NAME, DATASTORE_ID_SECOND)

In [16]:
def import_documents(
    project_id: str,
    location: str,
    data_store_id: str,
    source_documents: str,
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.DocumentServiceClient(client_options=client_options)

    # The full resource name of the search engine branch.
    # e.g. projects/{project}/locations/{location}/dataStores/{data_store_id}/branches/{branch}
    parent = client.branch_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        branch="default_branch",
    )

    request = discoveryengine.ImportDocumentsRequest(
        parent=parent,
        gcs_source=discoveryengine.GcsSource(
            input_uris=source_documents,
            data_schema="document",
        ),
        # Options: `FULL`, `INCREMENTAL`
        reconciliation_mode=discoveryengine.ImportDocumentsRequest.ReconciliationMode.FULL,
    )

    # Make the request
    operation = client.import_documents(request=request)

    response = operation.result()

    # Once the operation is complete,
    # get information from operation metadata
    metadata = discoveryengine.ImportDocumentsMetadata(operation.metadata)

    # Handle the response
    return operation.operation.name

## Create a Search Engine

This is used to set the `search_tier` to enterprise and to enable advanced LLM features.

Enterprise tier is required to get extractive answers from a search query and advanced LLM features are required to summarize search results.

In [11]:
def create_engine(
    project_id: str, location: str, engine_id: str, app_name: str, data_store_ids
):
    # Create a client
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = discoveryengine.EngineServiceClient(client_options=client_options)

    # Initialize request argument(s)
    engine = discoveryengine.Engine(
        display_name=app_name,
        solution_type=discoveryengine.SolutionType.SOLUTION_TYPE_SEARCH,
        industry_vertical=discoveryengine.IndustryVertical.GENERIC,
        data_store_ids=data_store_ids,
        search_engine_config=discoveryengine.Engine.SearchEngineConfig(
            search_tier=discoveryengine.SearchTier.SEARCH_TIER_ENTERPRISE,
            search_add_ons=[discoveryengine.SearchAddOn.SEARCH_ADD_ON_LLM],
        ),
    )

    request = discoveryengine.CreateEngineRequest(
        parent=client.collection_path(project_id, location, "default_collection"),
        engine=engine,
        engine_id=engine_id,
    )

    # Make the request
    operation = client.create_engine(request=request)
    response = operation.result(timeout=90)

In [12]:
#Creating engine usually take more than 5 minutes
import uuid
ENGINE_ID = str(uuid.uuid4())
create_engine(PROJECT_ID, LOCATION, ENGINE_ID, "Blended research", [DATASTORE_ID_FIRST, DATASTORE_ID_SECOND])

In [18]:
#Empty bq table took around 10 minutes
import_documents(PROJECT_ID, LOCATION, DATASTORE_ID_FIRST, [f'gs://{EXPORT_BUCKET}/{BQ_TABLE_1}.jsonl'])

['gs://jk-bq-export/sandbox-373102-first.jsonl']


'projects/1045259343465/locations/global/collections/default_collection/dataStores/pension_report_first-id/branches/0/operations/import-documents-7640641966568060229'

In [20]:
#Empty bq table took around 10 minutes
import_documents(PROJECT_ID, LOCATION, DATASTORE_ID, [f'gs://{EXPORT_BUCKET}/{BQ_TABLE_2}.jsonl'])

['gs://jk-bq-export/sandbox-373102-second.jsonl']


'projects/1045259343465/locations/global/collections/default_collection/dataStores/pension_report_second-id/branches/0/operations/import-documents-12276097102817183724'

## Query your Search Engine

Note: The Engine will take some time to be ready to query.

If you recently created an engine and you receive an error similar to:

`404 Engine {ENGINE_ID} is not found`

Then wait a few minutes and try your query again.

In [23]:
def search_sample(
    project_id: str,
    location: str,
    engine_id: str,
    search_query: str,
) -> list[discoveryengine.SearchResponse]:
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if LOCATION != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search engine serving config
    # e.g. projects/{project_id}/locations/{location}/dataStores/{data_store_id}/servingConfigs/{serving_config_id}
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_search"

    # Optional: Configuration options for search
    # Refer to the `ContentSearchSpec` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
        ),
    )

    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        content_search_spec=content_search_spec,
        #query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
        #    condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        #),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    return response

In [24]:
query = "국민연금 연도별 수익률"

response = search_sample(PROJECT_ID, LOCATION, ENGINE_ID, query)
print(response.summary.summary_text)

국민연금은 1988년 제도 시행 이후 33년간 가입자 2,211만 명, 수급자 559만 명, 기금적립금 834조 원의 규모로 성장했다. [1] 2020년 말 국민연금기금 적립금은 833조 7천억 원이며, 이중 대부분은 주식, 채권, 대체투자 등 금융자산에 투자된다. [1]

다음은 국민연금의 연도별 수익률 정보이다:

*   **2019년:**
    *   기금 전체 수익률은 11.31%이다. [3]
    *   금융부문 연간 수익률은 9.72%이며, 수익금은 72조 1천억 원이다. [1]
    *   국내 주식 시간가중수익률은 12.46%이다. [3]
    *   기금 설치 이후 2019년 말까지 연평균 누적 수익률은 5.86%이며, 누적 수익금은 총 367조 5천억 원이다. [3]
*   **2020년:**
    *   기금 전체 운용수익률은 9.70%이며, 수익금은 72조 1천억 원이다. [1]
    *   기금 설치 이후 연평균 누적수익률은 6.27%이며, 누적수익금은 439조 6천억 원이다. [1]
    *   금융부문 연간 수익률은 9.72%이며, 수익금은 72조 1천억 원이다. [1]
    *   금융 부문 시간가중수익률은 9.58%이다. [1]
    *   국내주식 수익률은 34.66%이다. [1]
    *   해외주식 수익률은 10.22%이다. [1]
*   **2021년:**
    *   연간 운용수익률은 10.77%이며, 수익금은 91조 2천억 원이다. [2]
    *   기금 설치 이후 2021년 말까지 연평균 누적수익률은 6.76%이며, 누적 수익금은 530조 8천억 원이다. [2]
    *   금융부문 시간가중수익률은 10.86%이다. [2]
    *   해외주식 수익률은 29.77%이다. [2]

2019년에 국내 주식에 투자하는 업종 중 제조업 투자 비중이 64.9%로 과반을 차지했다. [3] 2020년에는 국내채권 비중이 39.1%로 축소되었고, 해외투자 비중은 36.5%까지 확대되었다. [1] 2021년